In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Wine-Quality-Prediction\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Wine-Quality-Prediction'

In [4]:
from mlproject.constants import *

[2025-01-30 19:51:52,765 : INFO : __init__ : Logger has been set up successfully!]


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    max_samples: int
    max_features: str
    max_depth: str
    criterion: str
    bootstrap: bool
    target_column: str

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories
from mlproject import logger

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            max_samples = params.max_samples,
            max_features = params.max_features,
            max_depth = params.max_depth,
            criterion = params.criterion,
            bootstrap = params.bootstrap,
            target_column = schema.name
        )

        return model_trainer_config

In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Validate file paths
        if not os.path.exists(self.config.train_data_path):
            raise FileNotFoundError(f"Training data file not found at {self.config.train_data_path}")
        if not os.path.exists(self.config.test_data_path):
            raise FileNotFoundError(f"Testing data file not found at {self.config.test_data_path}")

        # Load the data
        train_data = np.load(self.config.train_data_path, allow_pickle=True)
        test_data = np.load(self.config.test_data_path, allow_pickle=True)

        logger.info(f"Loaded train data: type={type(train_data)}, shape={train_data.shape}")
        logger.info(f"Loaded test data: type={type(test_data)}, shape={test_data.shape}")

        # Split features and target
        train_x = train_data[:, :-1]  # All columns except the last one
        train_y = train_data[:, -1]   # Only the last column
        test_x = test_data[:, :-1]    # All columns except the last one
        test_y = test_data[:, -1]     # Only the last column

        logger.info(f"Training data shape: X={train_x.shape}, y={train_y.shape}")
        logger.info(f"Testing data shape: X={test_x.shape}, y={test_y.shape}")

        # Train the model
        logger.info("Initializing RandomForestClassifier...")
        classifier = RandomForestClassifier(n_estimators=self.config.n_estimators,
                                            min_samples_split=self.config.min_samples_split,
                                            min_samples_leaf=self.config.min_samples_leaf,
                                            max_samples=self.config.max_samples,
                                            max_features=self.config.max_features,
                                            max_depth=self.config.max_depth,
                                            criterion=self.config.criterion,
                                            bootstrap=self.config.bootstrap,
                                            random_state=42)
        classifier.fit(train_x, train_y)

        logger.info("Training the model...")

        # Save the trained model
        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(classifier, model_path)
        logger.info(f"Model saved successfully at {model_path}")

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-01-30 19:51:55,162 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-01-30 19:51:55,166 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-01-30 19:51:55,171 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-01-30 19:51:55,173 : INFO : common : created directory at: artifacts]
[2025-01-30 19:51:55,174 : INFO : common : created directory at: artifacts/model_trainer]
[2025-01-30 19:51:55,178 : INFO : 3146687685 : Loaded train data: type=<class 'numpy.ndarray'>, shape=(3064, 12)]
[2025-01-30 19:51:55,179 : INFO : 3146687685 : Loaded test data: type=<class 'numpy.ndarray'>, shape=(1022, 12)]
[2025-01-30 19:51:55,179 : INFO : 3146687685 : Training data shape: X=(3064, 11), y=(3064,)]
[2025-01-30 19:51:55,180 : INFO : 3146687685 : Testing data shape: X=(1022, 11), y=(1022,)]
[2025-01-30 19:51:55,181 : INFO : 3146687685 : Initializing RandomForestClassifier...]
[2025-01-30 19:51:57,479 : INFO : 3146687685 : Training the model..